# LatentSim!

In [1]:
import numpy as np
from latent_sim import LatentSim

In [2]:
ls = LatentSim()

In [3]:
hub = "/Users/afq/Google Drive/networks/training_water_slgc_logp/"

In [4]:
ls.load_model(hub+"Classifying_2,1,12,24,sigmoid",
             scale_file="data_files/water_iapw_logp_ranges.csv",logp=True)

INFO:tensorflow:Restoring parameters from /Users/afq/Google Drive/networks/training_water_slgc_logp/Classifying_2,1,12,24,sigmoid/final_variables


In [5]:
ls.i_q

<tf.Tensor 'i_q:0' shape=(?, 2) dtype=float32>

In [6]:
ls.o_x

<tf.Tensor 'decode:0' shape=(?, 4) dtype=float32>

In [7]:
ls.decode(np.array([[0.0,0.0]]))

array([[4.1367200e+02, 3.7433156e+05, 5.5091467e+02, 1.4695822e+06]],
      dtype=float32)

# Finding initial conditions

We know we only need 2 variables to specify a point on the surface. Most of the time we just want to provide a p and a T. 
However, the econder assumes that the input is on the manifold. 
The user could go through the effort of looking up rho and h, but that's a pain!
We want to let the user specify only 2 numbers and find the intersection on the manifold.

If we just try to plug in the result with "reasonable" fillers for the two unknowns we notice that we encoded onto the wrong point on the manifold:

In [8]:
q = ls.encode(np.array([[250.0, 1.0e3, ls.scale[0,2],ls.scale[0,3]]]))
print q
print ls.decode(q)

[[-0.31545323  0.2924902 ]]
[[2.833821e+02 1.167086e+03 5.176789e+02 1.154175e+06]]


This means:
\begin{equation}
T,p \neq D_{Tp}(E(T,p,?,?))
\end{equation}
Unfortunately this means we have to start solving the equations.
\begin{equation}
R(q) = D_{Tp}(q) - \{T^*,p^*\}
\end{equation}
with a good initial guess of
\begin{equation}
q^0 = E(T^*,p^*,?,?)
\end{equation}

In [9]:
ls.encode(
    np.array([[2.50000000e+02, 1.00000000e+03, 5.41351758e+02 ,1.35349597e+06]]) )

array([[-0.31545323,  0.2924902 ]], dtype=float32)

We can wrap this process up in a nice interface,

In [10]:
q0 = ls.find_point(T=250,p=1.0e3)

Found point at  [[ 2.4999997e+02  1.0000082e+03  9.3469458e+02 -6.1855800e+05]]  after  138  iterations.


Note that we're working in 32-bit precision, so that's as close as we're going to get!

# Building the equations

In [11]:
ls.build_dae()

In [12]:
ls.lhs

<tf.Tensor 'sub_5:0' shape=(?, 2) dtype=float32>

In [13]:
ls.rhs

<tf.Tensor 'add_12:0' shape=(?, 2) dtype=float32>

In [14]:
ls._vars['T_inf'].eval(session=ls._sess)

300.0

# Running a simulation

In [15]:
def schedule(t):
    if t<100.0:
        ls.set_params(T_inf=800,p_inf=1.0e2)
    elif t<200.0:
        ls.set_params(T_inf=800,p_inf=3.0e7)
    elif t<300.0:
        ls.set_params(T_inf=400,p_inf=3.0e7)
    elif t<400.0:
        ls.set_params(T_inf=400,p_inf=1.0e3)

In [19]:
ls.set_params(Dt=1.0)
q = ls.solve_a_time_step(q)
ls.decode(qi)

KeyError: 'D'